In [41]:
import pandas as pd
import numpy as np
import re
import tweepy
import json
from textblob import TextBlob
pd.set_option('display.max_colwidth', 280)
import time
from datetime import timedelta
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()

<IPython.core.display.Javascript object>

[nltk_data] Downloading package vader_lexicon to C:\Users\Intel PC
[nltk_data]     G5900\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [42]:
eng_twits = pd.read_csv('eng_twits.csv')
eng_twits['tweet_id'] = eng_twits['tweet_id'].apply(lambda x: x if pd.isnull(x) else str(x))
eng_twits['user_id'] = eng_twits['user_id'].apply(lambda x: x if pd.isnull(x) else str(x))

In [43]:
eng_twits.shape

(1450, 13)

In [71]:
def cleantxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text) #remove mentions
    text = re.sub(r"#(\w+)", ' ', text) #remove hashtag words
    text = re.sub(r'#', ' ', text) #removing the # symbol
    text = re.sub(r'RT[\s]+', ' ', text) #removing RT
    text = re.sub(r'https?:\/\/\S+', ' ', text) #removing the hyper link
    text = text.replace('\n', ' ')
    text = text.replace('_', ' ')
    text = text.strip()
    text = text.lower()
    return text

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def polarity(txt):
    twit_polarity = TextBlob(txt).sentiment.polarity
    return twit_polarity
def subjectivity(txt):
    twit_subjectivity = TextBlob(txt).sentiment.subjectivity
    return twit_subjectivity

def polarity_kind(num):
    if num > 0:
        polarity_kind = 'Positive'
    elif num == 0:
        polarity_kind = 'Neutral'
    elif num < 0:
        polarity_kind = 'Negative'
    return polarity_kind

def vader_polarity(txt):
    vader_polarity = vader.polarity_scores(txt)['compound']
    return vader_polarity

def vader_polarity_kind(num):
    if num > 0:
        vader_polarity_kind = 'Positive'
    elif num == 0:
        vader_polarity_kind = 'Neutral'
    elif num < 0:
        vader_polarity_kind = 'Negative'
    return vader_polarity_kind

In [72]:
eng_twits['cleaned_tweets'] = eng_twits['tweets'].apply(lambda x: x if pd.isnull(x) else cleantxt(x))
eng_twits['noemojis_tweets'] = eng_twits['cleaned_tweets'].apply(lambda x: x if pd.isnull(x) else remove_emojis(x))
eng_twits['twits_polarity'] = eng_twits['noemojis_tweets'].apply(lambda x: x if pd.isnull(x) else polarity(x))
eng_twits['twits_subjectivity'] = eng_twits['noemojis_tweets'].apply(lambda x: x if pd.isnull(x) else subjectivity(x))
eng_twits['polarity_kinds'] = eng_twits['twits_polarity'].apply(lambda x: x if pd.isnull(x) else polarity_kind(x))
eng_twits['twits_vader_polarity'] = eng_twits['noemojis_tweets'].apply(lambda x: x if pd.isnull(x) else vader_polarity(x))
eng_twits['vader_polarity_kinds'] = eng_twits['twits_vader_polarity'].apply(lambda x: x if pd.isnull(x) else vader_polarity_kind(x))


In [73]:
eng_twits.head()

,description,followers,following,hashtags,location,retweet_hashtags,retweetcount,time,totaltweets,tweet_id,tweets,user_id,username,cleaned_tweets,noemojis_tweets,twits_polarity,twits_subjectivity,polarity_kinds,twits_vader_polarity,vader_polarity_kinds
0,"❀Wherever you go,the crystal rises up and begins to fly❀",13.0,77.0,[],NaN,"['Πυρκαγια', 'PrayForGreece', 'greeceisburning']",627.0,2021-08-07 12:01:27,5829.0,1424102677370138624,"this is the real greece. more than 140 active wildfires. people evacuating, animals getting burnt alive. greece isn’t only santorini and mykonos. #Πυρκαγια #PrayForGreece #greeceisburning https://t.co/oCMhbWdp1g https://t.co/VbmNZr8Ytp",1116305803982409728,puppykittychimi,"this is the real greece. more than 140 active wildfires. people evacuating, animals getting burnt alive. greece isn’t only santorini and mykonos.","this is the real greece. more than 140 active wildfires. people evacuating, animals getting burnt alive. greece isn’t only santorini and mykonos.",0.133333,0.56,Positive,0.6771,Positive
1,ta tout partout #TwitterBot🤖 #TheGreenNation #FranceVerte🇫🇷 #LegalizeCannabis #EndTheWarOnDrugs #LegalizeIt🌱 #UnitedNations🇺🇳 #NationsUnies🇺🇳,7527.0,2076.0,"['ΛιμνηΕυβοιας', 'GreeceFires', 'Πυρκαγια']",Belgium 🇧🇪🇪🇺,"['ΛιμνηΕυβοιας', 'GreeceFires', 'Πυρκαγια', 'ΠυρκαγιαΕυβοια']",1.0,2021-08-07 20:07:13,820029.0,1424102674782461958,I'm just gonna leave this here. Evacuation of #ΛιμνηΕυβοιας. Straight out of a horror movie. #GreeceFires #Πυρκαγια #ΠυρκαγιαΕυβοια https://t.co/6NuovQ4fyJ,1129087959339327488,4Tchat,i'm just gonna leave this here. evacuation of . straight out of a horror movie.,i'm just gonna leave this here. evacuation of . straight out of a horror movie.,0.200000,0.40,Positive,-0.4588,Negative
2,i hope we can be friends ^^\n\n | rt heavy |\nic: @os_amaniwa \n\n\npity for yoimiya: (47/180) (3/80) (40/90)\nasia ar 56 | eu ar 23,871.0,928.0,[],she/her 16 asian,"['PrayForGreece', 'πυρκαγια', 'φωτια_ευβοια', 'βοηθεια']",49.0,2021-08-07 13:55:39,5277.0,1424102530670206976,"RT!! please. i’m begging😭💔\nmy country is being burned to the ground. i’m chronically ill &amp; disabled, &amp; take care of many animals. we need your help. for food, meds, water bottles, canned foods..\n\npaypal; its.joyfulkate@gmail.com\n\n#PrayForGreece #πυρκαγια #φωτια_ε...",1401852239623254019,unofficialyae,"rt!! please. i’m begging😭💔 my country is being burned to the ground. i’m chronically ill &amp; disabled, &amp; take care of many animals. we need your help. for food, meds, water bottles, canned foods.. paypal; its.joyfulkate .com","rt!! please. i’m begging my country is being burned to the ground. i’m chronically ill &amp; disabled, &amp; take care of many animals. we need your help. for food, meds, water bottles, canned foods.. paypal; its.joyfulkate .com",-0.066667,0.60,Negative,0.7170,Positive
3,I'm your hate when you want love,60.0,160.0,[],NaN,"['PrayForGreece', 'Πυρκαγια']",29.0,2021-08-07 12:32:00,2140.0,1424102493781467136,"Y'all dreaming about vacations in Greece posting pretty photos.. let me show you a ""pretty photo"".\nThis was captured by a Greek photographer on the national road of Lamia - Athens. It translates to: 'Greece, how sorry I am.'\nWe're living a holocaust.. #PrayForGreece #Πυρκαγ...",1104420710464843776,vagabondk8,"y'all dreaming about vacations in greece posting pretty photos.. let me show you a ""pretty photo"". this was captured by a greek photographer on the national road of lamia - athens. it translates to: 'greece, how sorry i am.' we're living a holocaust..","y'all dreaming about vacations in greece posting pretty photos.. let me show you a ""pretty photo"". this was captured by a greek photographer on the national road of lamia - athens. it translates to: 'greece, how sorry i am.' we're living a holocaust..",0.000000,0.75,Neutral,0.7269,Positive
4,♪,456.0,808.0,[],Greece🇬🇷 |She/her| ↑16↓,"['Πυρκαγια', 'PrayForGreece', 'greeceisburning']",627.0,2021-08-07 12:01:27,14687.0,14241024804

In [63]:
eng_twits[eng_twits['twits_vader_polarity']!=0].sort_values(['twits_polarity','retweetcount'], ascending=[True,False])

,description,followers,following,hashtags,location,retweet_hashtags,retweetcount,time,totaltweets,tweet_id,tweets,user_id,username,cleaned_tweets,noemojis_tweets,twits_polarity,twits_subjectivity,polarity_kinds,twits_vader_polarity
1196,Eoin || 29 || he/him || my interests are many and my time management is shit || @lkyjellyl 💍15/02/2020,35.0,629.0,"['Greece', 'φωτιές']",u g h,"['Greece', 'φωτιές', 'φωτιά', 'πυρκαγιά', 'εύβοια', 'Evoia', 'Evia']",6.0,2021-08-08 20:20:03,21207.0,1424629414852190211,6th day of fires. Thousands have now evacuated their homes. PIctures are just devastating (cc/ gettyImages). \n\n#Greece #φωτιές #φωτιά #πυρκαγιά #εύβοια #Evoia #Evia https://t.co/X9sTyGty7U https://t.co/B9sUwsOVTc,947380945371398145,snailespace,6th day of fires. thousands have now evacuated their homes. pictures are just devastating (cc/ gettyimages).,6th day of fires. thousands have now evacuated their homes. pictures are just devastating (cc/ gettyimages).,-1.0,1.0,Negative,-0.6486
1224,"gay, geordie, cats. specifically one-eyed blind #blackcat she/her 🟨🟥",765.0,593.0,"['Greece', 'φωτιές']",NaN,"['Greece', 'φωτιές', 'φωτιά', 'πυρκαγιά', 'εύβοια', 'Evoia', 'Evia']",6.0,2021-08-08 20:20:03,78323.0,1424620361723883521,6th day of fires. Thousands have now evacuated their homes. PIctures are just devastating (cc/ gettyImages). \n\n#Greece #φωτιές #φωτιά #πυρκαγιά #εύβοια #Evoia #Evia https://t.co/X9sTyGty7U https://t.co/B9sUwsOVTc,23185808,sarahdal,6th day of fires. thousands have now evacuated their homes. pictures are just devastating (cc/ gettyimages).,6th day of fires. thousands have now evacuated their homes. pictures are just devastating (cc/ gettyimages).,-1.0,1.0,Negative,-0.6486
1018,"Dad, Husband, Citizen, Veteran, header compliments of @ArchAngLJabroni\n\nhttps://t.co/veWqBCzwyR",17739.0,9353.0,[],NaN,"['greeceisburning', 'PrayForGreece', 'wildfires', 'Ευβοια', 'φωτιες', 'Πυρκαγια']",4.0,2021-08-07 19:28:25,237839.0,1424104574646882309,The fires in Greece are devastating. Their firefighters are under-equipped to handle them. It's devastating and heartbreaking. \nLink in my bio to help if you can. 100% of donations goes to relief efforts. \n#greeceisburning #PrayForGreece #wildfires #Ευβοια #φωτιες #Πυρκαγια...,1079182332731576325,youjustroll,the fires in greece are devastating. their firefighters are under-equipped to handle them. it's devastating and heartbreaking. link in my bio to help if you can. 100% of donations goes to relief efforts.,the fires in greece are devastating. their firefighters are under-equipped to handle them. it's devastating and heartbreaking. link in my bio to help if you can. 100% of donations goes to relief efforts.,-1.0,1.0,Negative,-0.7783
235,🖤🎼🎧💦🌊💙,1624.0,411.0,[],NaN,"['greeceisburning', 'PrayForGreece', 'wildfires', 'Ευβοια', 'φωτιες', 'Πυρκαγια']",3.0,2021-08-07 19:28:25,81668.0,1424091015649144835,The fires in Greece are devastating. Their firefighters are under-equipped to handle them. It's devastating and heartbreaking. \nLink in my bio to help if you can. 100% of donations goes to relief efforts. \n#greeceisburning #PrayForGreece #wildfires #Ευβοια #φωτιες #Πυρκαγια...,1098100514334695424,xaoskeno,the fires in greece are devastating. their firefighters are under-equipped to handle them. it's devastating and heartbreaking. link in my bio to help if you can. 100% of donations goes to relief efforts.,the fires in greece are devastating. their firefighters are under-equipped to handle them. it's devastating and heartbreaking. link in my bio to help if you can. 100% of donations goes to relief efforts.,-1.0,1.0,Negative,-0.7783
242,NaN,5.0,53.0,[],"Αττική, Ελλάς","['greeceisburning', 'PrayForGreece', 'wildfires', 'Ευβοια', 'φωτιες', 'Πυρκαγια']",3.0,2021-08-07 19:28:25,256.0,1424090517936254977,The fires in Greece are devastating. Their firefighters are under-equipped to handle them. It's devastating and heartbreaking. \nLink in my bio to help if you can. 100% of donations goes to relief effor

In [67]:
eng_twits[(eng_twits['twits_vader_polarity']!=0) & (eng_twits['twits_vader_polarity'].notna())]['twits_polarity'].mean()

0.12103785026155432

In [68]:
eng_twits['polarity_kinds'].value_counts()

Positive    892
Neutral     298
Negative    178
Name: polarity_kinds, dtype: int64

In [48]:
eng_twits['twits_polarity'].mean()

0.12970357522845583

In [49]:
eng_twits['twits_subjectivity'].mean()

0.47136775904909645

In [51]:
eng_twits['twits_vader_polarity'].mean()

0.30199751461988317

In [74]:
eng_twits.to_csv('eng_twits_analyzed.csv', index=False)

In [75]:
eng_twits_analyzed = pd.read_csv('eng_twits_analyzed.csv')

In [76]:
eng_twits_analyzed.head()

,description,followers,following,hashtags,location,retweet_hashtags,retweetcount,time,totaltweets,tweet_id,tweets,user_id,username,cleaned_tweets,noemojis_tweets,twits_polarity,twits_subjectivity,polarity_kinds,twits_vader_polarity,vader_polarity_kinds
0,"❀Wherever you go,the crystal rises up and begins to fly❀",13.0,77.0,[],NaN,"['Πυρκαγια', 'PrayForGreece', 'greeceisburning']",627.0,2021-08-07 12:01:27,5829.0,1424102677370138624,"this is the real greece. more than 140 active wildfires. people evacuating, animals getting burnt alive. greece isn’t only santorini and mykonos. #Πυρκαγια #PrayForGreece #greeceisburning https://t.co/oCMhbWdp1g https://t.co/VbmNZr8Ytp",1116305803982409728,puppykittychimi,"this is the real greece. more than 140 active wildfires. people evacuating, animals getting burnt alive. greece isn’t only santorini and mykonos.","this is the real greece. more than 140 active wildfires. people evacuating, animals getting burnt alive. greece isn’t only santorini and mykonos.",0.133333,0.56,Positive,0.6771,Positive
1,ta tout partout #TwitterBot🤖 #TheGreenNation #FranceVerte🇫🇷 #LegalizeCannabis #EndTheWarOnDrugs #LegalizeIt🌱 #UnitedNations🇺🇳 #NationsUnies🇺🇳,7527.0,2076.0,"['ΛιμνηΕυβοιας', 'GreeceFires', 'Πυρκαγια']",Belgium 🇧🇪🇪🇺,"['ΛιμνηΕυβοιας', 'GreeceFires', 'Πυρκαγια', 'ΠυρκαγιαΕυβοια']",1.0,2021-08-07 20:07:13,820029.0,1424102674782461958,I'm just gonna leave this here. Evacuation of #ΛιμνηΕυβοιας. Straight out of a horror movie. #GreeceFires #Πυρκαγια #ΠυρκαγιαΕυβοια https://t.co/6NuovQ4fyJ,1129087959339327488,4Tchat,i'm just gonna leave this here. evacuation of . straight out of a horror movie.,i'm just gonna leave this here. evacuation of . straight out of a horror movie.,0.200000,0.40,Positive,-0.4588,Negative
2,i hope we can be friends ^^\n\n | rt heavy |\nic: @os_amaniwa \n\n\npity for yoimiya: (47/180) (3/80) (40/90)\nasia ar 56 | eu ar 23,871.0,928.0,[],she/her 16 asian,"['PrayForGreece', 'πυρκαγια', 'φωτια_ευβοια', 'βοηθεια']",49.0,2021-08-07 13:55:39,5277.0,1424102530670206976,"RT!! please. i’m begging😭💔\nmy country is being burned to the ground. i’m chronically ill &amp; disabled, &amp; take care of many animals. we need your help. for food, meds, water bottles, canned foods..\n\npaypal; its.joyfulkate@gmail.com\n\n#PrayForGreece #πυρκαγια #φωτια_ε...",1401852239623254019,unofficialyae,"rt!! please. i’m begging😭💔 my country is being burned to the ground. i’m chronically ill &amp; disabled, &amp; take care of many animals. we need your help. for food, meds, water bottles, canned foods.. paypal; its.joyfulkate .com","rt!! please. i’m begging my country is being burned to the ground. i’m chronically ill &amp; disabled, &amp; take care of many animals. we need your help. for food, meds, water bottles, canned foods.. paypal; its.joyfulkate .com",-0.066667,0.60,Negative,0.7170,Positive
3,I'm your hate when you want love,60.0,160.0,[],NaN,"['PrayForGreece', 'Πυρκαγια']",29.0,2021-08-07 12:32:00,2140.0,1424102493781467136,"Y'all dreaming about vacations in Greece posting pretty photos.. let me show you a ""pretty photo"".\nThis was captured by a Greek photographer on the national road of Lamia - Athens. It translates to: 'Greece, how sorry I am.'\nWe're living a holocaust.. #PrayForGreece #Πυρκαγ...",1104420710464843776,vagabondk8,"y'all dreaming about vacations in greece posting pretty photos.. let me show you a ""pretty photo"". this was captured by a greek photographer on the national road of lamia - athens. it translates to: 'greece, how sorry i am.' we're living a holocaust..","y'all dreaming about vacations in greece posting pretty photos.. let me show you a ""pretty photo"". this was captured by a greek photographer on the national road of lamia - athens. it translates to: 'greece, how sorry i am.' we're living a holocaust..",0.000000,0.75,Neutral,0.7269,Positive
4,♪,456.0,808.0,[],Greece🇬🇷 |She/her| ↑16↓,"['Πυρκαγια', 'PrayForGreece', 'greeceisburning']",627.0,2021-08-07 12:01:27,14687.0,14241024804

In [17]:
vader.polarity_scores('prayforgreece')

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}